In [ ]:
#default_exp datasets

# Basic dataset

In [ ]:
#exporti
import torch

from dl4to.problem import Problem
from dl4to.datasets import TopoDataset

In [ ]:
#exporti
class CreateBasicProblems:
    @staticmethod
    def _get_blanks(size, resolution, dtype):
        shape = torch.round(resolution * size).type(torch.int)
        h = size / shape
        F = torch.zeros(3, *shape, dtype=dtype, requires_grad=False)
        Ω_dirichlet = torch.zeros(3, *shape, dtype=torch.bool, requires_grad=False)
        Ω_design = - torch.ones(1, *shape, dtype=int, requires_grad=False)
        return shape, h, F, Ω_dirichlet, Ω_design


    @staticmethod
    def _transform_force_per_area_to_force_per_volume(F, h):
        F[0, ...] /= h[0]
        F[1, ...] /= h[1]
        F[2, ...] /= h[2]
        return F


    @staticmethod
    def _transform_point_force_to_force_per_volume(F, h):
        return F / (h[0] * h[1] * h[2])


    @staticmethod
    def _get_padded_tensor(tensor, padding_depth=2):
        p_d = int(padding_depth)
        if p_d == 0:
            return tensor

        shape = tensor.shape
        assert len(shape) == 4
        padded_tensor = torch.zeros(shape[0], shape[1]+2*p_d, shape[2]+2*p_d, shape[3]+2*p_d, dtype=tensor.dtype)
        padded_tensor[:, p_d:-p_d, p_d:-p_d, p_d:-p_d] = tensor
        return padded_tensor


    @staticmethod
    def _tensile_rod(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'tensile_rod'
        size = torch.tensor([.1, .1, 1.])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, 0] = force_per_area
        Ω_dirichlet[:, :, :, -1] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _offset_tensile_rod(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'offset_tensile_rod'
        size = torch.tensor([.1, .1, 1.])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, 4] = force_per_area
        Ω_dirichlet[:, :, :, -4] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _inverse_tensile_rod(resolution=20, force_per_area=1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'inverse_tensile_rod'
        size = torch.tensor([.1, .1, 1.])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, :, :, 0] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _ledge(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'ledge'
        size = torch.tensor([1., .1, .2])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, 0, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:,-2] = 1.
        Ω_design[:,1,:,:] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _cube_ledge(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'cube_ledge'
        size = torch.tensor([.1, .1, 1.])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, 0, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:,-2] = 1.
        Ω_design[:,1,:,:] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _fork(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'fork'
        size = torch.tensor([1., .1, .2])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, 0:2, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:,-2] = 1.
        Ω_design[:,:,:,:2] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _inverse_fork(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'fork'
        size = torch.tensor([1., .1, .2])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, 0] = -force_per_area
        Ω_dirichlet[:, 0:2, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:, 1] = 1.
        Ω_design[:,:,:,-2:] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _cantilever(resolution=20, point_force=-1e7, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'cantilever'
        size = torch.tensor([1., .1, .3])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        center_y = int(shape[1]/2)
        F[-1, -1, center_y, 0] = point_force
        Ω_dirichlet[:, 0, :, :] = 1
        F = CreateBasicProblems._transform_point_force_to_force_per_volume(F, h)
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _supported_ledge(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'ledge'
        size = torch.tensor([1., .1, .2])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, 0, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:,-2] = 1.
        Ω_design[:,1,:,:] = 1.
        Ω_design[:,-3:-1,:,:] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _supported_unconnected_ledge(resolution=20, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'ledge'
        size = torch.tensor([1., .1, .2])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        F[-1, :, :, -1] = force_per_area
        Ω_dirichlet[:, 0, :, :] = 1
        F = CreateBasicProblems._transform_force_per_area_to_force_per_volume(F, h)
        Ω_design[:,:,:,-2] = 1.
        Ω_design[:,1,:,:] = 1.
        Ω_design[:,-3:-1,:,1:] = 1.
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)


    @staticmethod
    def _wheel(resolution=20, point_force=-2.5e6, E=7e10, ν=.3, σ_ys=4.5e8, dtype=torch.float32):
        name = 'wheel'
        size = torch.tensor([1., .4, .4])
        shape, h, F, Ω_dirichlet, Ω_design = CreateBasicProblems._get_blanks(size, resolution, dtype)
        center = shape / 2
        center0, center1 = int(center[0]), int(center[1])
        F[-1, center0-1:center0+1, center1-1:center1+1, 0] = point_force/4
        F = CreateBasicProblems._transform_point_force_to_force_per_volume(F, h)
        Ω_dirichlet[:, 0:2, :, -1] = 1
        Ω_dirichlet[:, -3:-1, :, -1] = 1
        Ω_design[0 != (torch.sum(Ω_dirichlet + (F != 0), dim=0, keepdim=True))] = 1
        return Problem(E=E, ν=ν, σ_ys=σ_ys, h=h, Ω_dirichlet=Ω_dirichlet,
                       Ω_design=Ω_design, F=F, name=name, dtype=dtype)

In [ ]:
#export
class BasicDataset(TopoDataset):
    """
    A dataset that contains several basic problems that we found were common problems in the literature. The dataset itself contains no problems directly, but they can be individually accessed, 
    via their respecive function calls, e.g., `dataset = BasicDataset().ledge()`. The dataset contains no ground truth solutions.
    """
    def __init__(self, 
                 resolution:int=20, # The number of voxels of the dimension with the most voxels.
                 verbose:bool=True, # Whether to give the user feedback on the progress.
                 dtype:torch.dtype=torch.float32 # The datatype of the tensors that define the problem.
                ):
        self.resolution = resolution
        self._dtype = dtype
        super().__init__(verbose=verbose)


    @property
    def dtype(self):
        return self._dtype


    def tensile_rod(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._tensile_rod(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def offset_tensile_rod(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._offset_tensile_rod(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def inverse_tensile_rod(self, force_per_area=1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._inverse_tensile_rod(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def ledge(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._ledge(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def cube_ledge(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._cube_ledge(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def fork(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._fork(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def inverse_fork(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._inverse_fork(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def cantilever(self, point_force=-1e7, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._cantilever(resolution=self.resolution, point_force=point_force, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def supported_ledge(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._supported_ledge(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def supported_unconnected_ledge(self, force_per_area=-1.5e5, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._supported_unconnected_ledge(resolution=self.resolution, force_per_area=force_per_area, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)


    def wheel(self, point_force=-2.5e6, E=7e10, ν=.3, σ_ys=4.5e8):
        return CreateBasicProblems()._wheel(resolution=self.resolution, point_force=point_force, E=E, ν=ν, σ_ys=σ_ys, dtype=self.dtype)

In [ ]:
%%time
#hide

def test_resolution_argument(resolution):
    dataset = BasicDataset(resolution=resolution)
    assert dataset.resolution == resolution


test_resolution_argument(resolution=10)
test_resolution_argument(resolution=20)

CPU times: user 88 µs, sys: 61 µs, total: 149 µs
Wall time: 157 µs


In [ ]:
%%time
#hide

def test_that_we_can_get_ledge_cantilever_and_wheel(resolution):
    ledge      = BasicDataset(resolution=resolution).ledge()
    cantilever = BasicDataset(resolution=resolution).cantilever()
    wheel      = BasicDataset(resolution=resolution).wheel()

    assert max(ledge.shape) == resolution
    assert max(cantilever.shape) == resolution
    assert max(wheel.shape) == resolution


test_that_we_can_get_ledge_cantilever_and_wheel(resolution=10)
test_that_we_can_get_ledge_cantilever_and_wheel(resolution=20)

CPU times: user 14.1 ms, sys: 8.59 ms, total: 22.7 ms
Wall time: 37.2 ms
